1. 네이버증권 뉴스 본문 DB에서 가져와서 토큰화, 벡터화 하기
- 테이블 분리 10등분
- DB에서 본문 텍스트 summary 읽어오기
- konlpy로 토큰화 하기
- 이미 생성한 2만 단어 사전 으로 백터화
##* 코랩실행시 우선 konlpy 종목명으로 업데이트하기

In [ ]:
# 코랩에서 실행시 pyMySQL을 먼저 설치해야 한다
# !pip install JPype1
# !pip install konlpy
# !pip install pyMySQL
# !pip install -U finance-datareader

In [1]:
# import packages
import os
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
import konlpy
from konlpy.tag import Okt
import pymysql
from os import replace
import requests
from tqdm.notebook import tqdm
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import ast
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
import re
import string
# 리스트 flatten을 위한 itertools 패키지 import
import itertools
import warnings
warnings.filterwarnings('ignore')

2. konlpy에 종목명과 약명을 새로운 단어로 추가하기

In [ ]:
# --konlpy 커스터마이징  ----------------------------------------------
# --------------------------------------------------------------------
# Konlpy에 한국거래소 상장종목 전체 종목명과 약명을 새로운 단어로 추가하기
# --------------------------------------------------------------------
# konlpy 디렉토리로 이동
os.chdir('konlpy/설치된/디렉토리/java')
# 현재 디렉토리 확인
os.getcwd()
# 압축 풀 임시 디렉토리 /temp 만들기
os.makedirs('./temp')
# 임시디렉토리로 이동
os.chdir('./temp')
# 압축 풀기
!jar xvf ../open-korean-text-2.1.0.jar

# FinanceDataReader로 한국거래소 상장종목 전체 종목을 가져오기
df_krx = fdr.StockListing('KRX')
# 종목명만 추출하기
name_list = df_krx['Name']
# 새로운 단어목록 생성을 위한 data 변수 선언
data = ''
# 종목명을 한줄씩(\n)씩 넣어서 목록 만들기
for one_name in name_list:
  data += one_name+'\n'
# 종목명 약명 새로운 단어로 넣기
# DB 연결 준비: 종목명 약어 읽어오기
conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
# DB에서 종목병 약어 추출위한 sql 준비
sql = f"SELECT x.kor_name,x.kor_abb FROM DB이름.테이블이름 x"
# DB 검색결과를 dataframe에 저장
name_abb_df = pd.read_sql_query(sql, conn)
# 실행확인을 위한 화면 출력
print(name_abb_df.head(3))
# DB close
conn.close()
# 종목명 추가하기
name_list = name_abb_df['kor_name']
# 종목명을 한줄씩(\n)씩 넣어서 목록 만들기
for one_name in name_list:
  data += one_name+'\n'
# 종목명 약명 추가하기
name_list = name_abb_df['kor_abb']
# 종목명 약명을 한줄씩(\n)씩 넣어서 목록 만들기
for one_name in name_list:
  data += one_name+'\n'
# konlpy의 company_names.txt 사전으로 저장
with open("konlpy/설치된/디렉토리/java/temp/org/openkoreantext/processor/util/noun/company_names.txt", 'w') as f:
    f.write(data)
# 임시 작업 디렉토리로 이동 확인
os.chdir('konlpy/설치된/디렉토리/java/temp')
# 다시 압축
!jar cvf ../open-korean-text-2.1.0.jar *
# 코랩 실행시 Restart session 실행

# --konlpy 커스터마이징 END --------------------------------------------

* DB 에서 뉴스 본문 summary 전체 읽어오기

In [22]:
# DB 연결 준비: 뉴스 본문 읽어오기
conn = pymysql.connect(host= '호스트 주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
# DB에서 뉴스(summary, press, rdate) 추출위한 sql 준비
sql = f"SELECT x.article_id, x.r_time, x.summary FROM DB이름.테이블이름 x"
# DB 검색결과를 dataframe에 저장
result_df = pd.read_sql_query(sql, conn)
# 실행확인을 위한 화면 출력
#print(result_df.head(3))
# DB close
conn.close()

3. 뉴스 본문 summary를 10등분해서 테이블 나눠서 DB 저장
- 테이블 나누기: 너무 크면 load 가 안됨
- Tokenize해서 DB에 저장하기
- konlpy의 okt를 사용하여 형태소 분석하고 tokenize한 후 DB에 저장
- DB: fast_news_url, 테이블명: bonmoon_1_10 ~ bonmoon_10_10

In [ ]:
# 전체 데이터 수: 약 24만개
print(len(result_df))
# 날짜순으로 정렬
result_df= result_df.sort_values('r_time')
# 데이터프레임의 전체 행 수 구하기
num_rows = len(result_df)
# 전체를 10등분으로 나누기위한 행수 구하기
split_size = num_rows // 10
# 나눈 결과 저장을 위한 리스트
slices = []
# 데이터프레임을 10등분으로 나누기: 슬라이싱
for i in range(0, num_rows, split_size):
    slices.append(result_df.iloc[i:i+split_size])
# 10개의 테이블로 나눠서 DB에 저장
for i in range(10):
  # 임시 df
  temp_df = slices[i]
  # DB 저장을 위한 연결 준비
  db_connection_path = 'mysql+pymysql://아이디:패스워드@호스트주소:포트번호/DB이름'
  db_connection = create_engine(db_connection_path)
  # 데이터프레임을 DB로 저장, bonmoon_{번호}_10를 테이블 명으로, 이미 있으면 덮어쓰기(replace)
  temp_df.to_sql('bonmoon_'+str(i+1)+'_10',con=db_connection,if_exists='replace',index=False)
  # DB close
  db_connection.dispose()


In [25]:
# 한개의 테이블에 약 2만4천개
print(len(slices[0]))

24003


* 토큰화,인코딩,DB 저장

In [28]:
# -- custom_standardization_fn() --------------------------------------------
# custom_표준화 함수
def custom_standardization_fn(string_tensor):
    lowercase_string = string_tensor # 영문의 경우 여기서 모두 소문자 처리, 한글은 필요없음
    return tf.strings.regex_replace(
        # 문장부호 처리: %와 .는 살리고 나머지는 제외
        lowercase_string, f"[{re.escape(string.punctuation.replace('%','').replace('.',''))}]", "") 
# -- custom_standardization_fn() :END----------------------------------------

# -- custom_split_fn()-------------------------------------------------------
# custom_토큰화 함수
def custom_split_fn(string_tensor):
    return tf.strings.split(string_tensor)  # 한글의 경우 형태소 분석 처리 해야 한다
# -- custom_split_fn() :END--------------------------------------------------

# --tokenize_save_bonmoon()--------------------------------------------------
# 토큰화 해서 DB 저장
def tokenize_save_bonmoon(bonmoon_df,part_num):
  # 원본 copy 생성
  result_df = bonmoon_df.copy()
  # konlpy의 okt 생성
  okt=Okt()
  # 실행확인을 위한 화면 출력
  print(part_num)
  
  # okt로 tokenize해서 형태소 품사 morph 추출하기 (pos 는 okt.pos, 명사는 okt.nouns)
  result_df['morphs_tokenized_summary'] = result_df['summary'].apply(okt.morphs)
  # DB 저장을 위해 문자열로 변환
  result_df['morphs_tokenized_summary'] = result_df['morphs_tokenized_summary'].astype(str)

  # okt로 tokenize해서 형태소 pos 추출하기 (품사 전체는 okt.morphs, 명사는 okt.nouns)
  result_df['pos_tokenized_summary'] = result_df['summary'].apply(okt.pos)
  
  # DB 저장을 위해 문자열로 변환
  result_df['pos_tokenized_summary'] = result_df['pos_tokenized_summary'].astype(str)

  # okt로 tokenize해서 명사 형태소 nouns 추출하기 (품사 전체는 okt.morphs)
  result_df['nouns_tokenized_summary'] = result_df['summary'].apply(okt.nouns)
  
  # DB 저장을 위해 문자열로 변환
  result_df['nouns_tokenized_summary'] = result_df['nouns_tokenized_summary'].astype(str)

  # pos tokenized 에서 Modifier,Josa,Suffix,Punctuation,Foreigh 제외, 1 자리 단어 제외
  result_df['tokenized_summary'] = ""
  for i in range(len(result_df)):
    # 새로운 문자열 위한 리스트
    new_str = []
    # 튜플 형태로 꺼내면 (단어,품사) 형태이므로 뒤세 품사를 보고 맞는 단어들만 새로운 문자열로 만든다
    for (x,y) in ast.literal_eval(result_df['pos_tokenized_summary'][i]):
      # 관형사,조사,접두사,문장부호,한자및 기타기호 제외
      if (y!='Modifier')&(y!='Josa')&(y!='Suffix')&(y!='Punctuation')&(y!='Foreign'):
        # 한글자 짜리 제외
        if(len(str(x))>1):
          new_str.append(x)
    # 실행 확인을 위한 화면 출력
    #print(new_str)

    # 토크나이즈된 문자열 데이터프레임에 넣기
    result_df['tokenized_summary'][i]=str(new_str)

  # custom 표준화, 토큰화 함수 지정해서 TextVectoriztion 생성하기
  text_vectorization = TextVectorization(
      # max_tokens 지정
      max_tokens=20000
      # 정수 인덱스로 출력
    ,  output_mode="int"
      # 표준화 함수를 custom_표준화 함수로 지정
    ,  standardize=custom_standardization_fn
      # 토큰화 함수를 custom_토큰화 함수로 지정
    ,  split=custom_split_fn
  )
  #-------------------------------------------------------------------
  # --- 2만단어사전 불러오기 load : start ---
  from_disk = pickle.load(open("사전경로/tv_layer_20000.pkl", "rb"))

  new_vectorizer = TextVectorization(
      # max_tokens 지정
      max_tokens=20000
      # 정수 인덱스로 출력
    ,  output_mode="int"
      # 표준화 함수를 custom_표준화 함수로 지정
    ,  standardize=custom_standardization_fn
      # 토큰화 함수를 custom_토큰화 함수로 지정
    ,  split=custom_split_fn
    )
  new_vectorizer.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
  new_vectorizer.set_weights(from_disk['weights'])
  # --- 2만단어사전 불러오기 load : end ---
  
  # 인코딩하기
  # 위 생성한 백터라이즈를 하루치 속보에 적용 
  result_df['20000_encoded_text']= list(map(new_vectorizer,result_df['tokenized_summary']))
  # 하루에 여러 건 뉴스가 있으니 모든 뉴스에 적용 
  for i in range(len(result_df)):
    # 숫자로 인코드된 뉴스를 문자열로 만들어서 DB에 저장
    result_df.at[i,'20000_encoded_text_code']= str(new_vectorizer(result_df.at[i,'tokenized_summary']).numpy()).replace('\n', '')
  # DB 저장하기 위한 컬럼정리
  result_df = result_df[['article_id','r_time','summary','pos_tokenized_summary','tokenized_summary','20000_encoded_text_code']]
  # DB 저장을 위한 연결 준비
  db_connection_path = 'mysql+pymysql://아이디:패스워드@호스트주소:포트번호/DB이름'
  db_connection = create_engine(db_connection_path)
  # 데이터프레임을 DB로 저장, 날짜 ddate_sokbo를 테이블 명으로, 이미 있으면 덮어쓰기(replace)
  result_df.to_sql('bonmoon_'+str(part_num)+'_10',con=db_connection,if_exists='replace',index=False)
  # DB close
  db_connection.dispose()


In [ ]:
#====================================================
# 뉴스 본문 summary 인코딩하기: 메인 함수 호출
#----------------------------------------------------
# 10개 테이블 순차적으로 처리
for i in range(1,10):
  # 테이블 일련번호
  part_num = i+1
  # DB 연결 준비: 뉴스 본문 읽어오기(5개 테이블)
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # DB에서 뉴스(summary, press, rdate) 추출위한 sql 준비
  sql = f"SELECT x.*  FROM DB이름.bonmoon_{part_num}_10 x"
  # DB 검색결과를 dataframe에 저장
  part_df = pd.read_sql_query(sql, conn)
  # DB close
  conn.close()
  # 실행확인을 위한 화면 출력
  print(part_df.head(3))
  #== 메인함수 호출 =========================
  tokenize_save_bonmoon(part_df, part_num)
  #=========================================
